In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [2]:
# using the SQLite Table to read data.
con = sqlite3.connect('database.sqlite') 
#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews """, con) 


# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 0
    elif x == 3:
        return 1
    else:
        return 2

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (568454, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,2,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,2,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


#  Exploratory Data Analysis

## Data Cleaning: Deduplication


In [3]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


As can be seen above the same user has multiple reviews of the with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text  and on doing analysis it was found that <br>
<br> 
ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)<br>
<br> 
ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on<br>

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [4]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [5]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(393933, 10)

In [6]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.29901100176971

It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions
## Data Cleaning (contd..)

In [7]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Id=44737 OR Id=64422
ORDER BY ProductID
""", con)

display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [8]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [9]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(393931, 10)


2    307052
0     57107
1     29772
Name: Score, dtype: int64

In [10]:
score = list(final['Score'])
print(type(score))
print(len(score))

<class 'list'>
393931


In [11]:
outfile = open("score.pkl", "wb")
pickle.dump(score, outfile)

## Text Preprocessing: Stemming, stop-word removal and Lemmatization.

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [12]:
# find sentences containing HTML tags
import re
i=0;
for sent in final['Text'].values:
    if (len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i += 1;

0
In June<br />I saw a charming group<br />of roses all begin<br />to droop<br />I pepped them up<br />with chicken soup!<br />Sprinkle once<br />sprinkle twice<br />sprinkle chicken soup<br />with rice<br /><br />This is a great book to teach children the months of the year. The repetition of the phrases and the funny little stories with accompanying pictures make for an ideal bedtime read. This isn't nearly as good as some of Sendak's other books (like Where the Wild Things are or Pierre: The Boy Who Didn't Care), but it still carries his unique brand of charm.


In [14]:

stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))

{'those', "shan't", 'hadn', 'whom', 'very', 'aren', 'an', 'mightn', 'himself', 'was', 'shouldn', "didn't", 'few', 'but', 'his', 'them', 'with', 'our', 'above', 'she', 'herself', 'have', 'couldn', "shouldn't", 'through', 'off', "should've", "you're", 'at', 'into', 'don', 'me', 'they', 'against', 'are', 'because', 'down', 'same', 'each', 'we', 'm', 'for', "mustn't", 'wouldn', 'on', 'below', 'been', 'your', 'am', 'again', 'during', 'll', "doesn't", 'some', "mightn't", 'yourselves', "needn't", 'over', 'should', 'the', 'and', "won't", 't', 'if', 're', 'ain', 'nor', "weren't", 'her', 'doing', 'isn', 'a', "wouldn't", 'of', 'any', 'you', "hasn't", 'my', 'being', 'from', 'why', "aren't", 'ourselves', 'o', 'won', 'only', 'were', 'after', 'having', 'then', 'ours', 'by', 'mustn', 'has', 'both', 'had', 'so', 'before', 'now', 'what', 'can', 'theirs', 'yourself', 'its', 'until', 'while', "it's", 'where', "you'd", 'it', "wasn't", 'to', 'do', 'myself', 'further', 's', 'themselves', 'hers', "isn't", 'we

In [17]:
#Code for implementing step-by-step the checks mentioned in the pre-processing phase
# this code takes a while to run as it needs to run on 500k sentences.
if not os.path.isfile('final.sqlite'):
    final_string=[]
    all_positive_words=[] # store words from +ve reviews here
    all_negative_words=[] # store words from -ve reviews here.
    all_neutral_words=[] # store words from neutralreviews here.
    for i, sent in enumerate(tqdm(final['Text'].values)):
        filtered_sentence=[]
        #print(sent);
        sent=cleanhtml(sent) # remove HTMl tags
        for w in sent.split():
            # we have used cleanpunc(w).split(), one more split function here because consider w="abc.def", cleanpunc(w) will return "abc def"
            # if we dont use .split() function then we will be considring "abc def" as a single word, but if you use .split() function we will get "abc", "def"
            for cleaned_words in cleanpunc(w).split():
                if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                    if(cleaned_words.lower() not in stop):
                        s=(sno.stem(cleaned_words.lower())).encode('utf8')
                        filtered_sentence.append(s)
                        if (final['Score'].values)[i] == 2: 
                            all_positive_words.append(s) #list of all words used to describe positive reviews
                        if(final['Score'].values)[i] == 0:
                            all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                        if(final['Score'].values)[i] == 1:
                            all_neutral_words.append(s)
        str1 = b" ".join(filtered_sentence) #final string of cleaned words
        #print("***********************************************************************")
        final_string.append(str1)

    #############---- storing the data into .sqlite file ------########################
    final['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 
    final['CleanedText']=final['CleanedText'].str.decode("utf-8")
        # store final table into an SQlLite table for future.
    conn = sqlite3.connect('final.sqlite')
    c=conn.cursor()
    conn.text_factory = str
    final.to_sql('Reviews', conn,  schema=None, if_exists='replace', \
                 index=True, index_label=None, chunksize=None, dtype=None)
    conn.close()
    
    
    with open('positive_words.pkl', 'wb') as f:
        pickle.dump(all_positive_words, f)
    with open('negitive_words.pkl', 'wb') as f:
        pickle.dump(all_negative_words, f)
    with open('neutral_words.pkl', 'wb') as f:
        pickle.dump(all_neutral_words, f)



  0%|          | 0/393931 [00:00<?, ?it/s]

  0%|          | 15/393931 [00:00<47:09, 139.24it/s]

  0%|          | 30/393931 [00:00<46:40, 140.64it/s]

  0%|          | 53/393931 [00:00<41:33, 157.95it/s]

  0%|          | 66/393931 [00:00<45:58, 142.78it/s]

  0%|          | 89/393931 [00:00<42:25, 154.70it/s]

  0%|          | 105/393931 [00:00<42:20, 155.02it/s]

  0%|          | 125/393931 [00:00<39:31, 166.08it/s]

  0%|          | 145/393931 [00:00<37:32, 174.86it/s]

  0%|          | 163/393931 [00:00<37:21, 175.65it/s]

  0%|          | 186/393931 [00:01<34:45, 188.76it/s]

  0%|          | 206/393931 [00:01<35:20, 185.70it/s]

  0%|          | 230/393931 [00:01<33:30, 195.83it/s]

  0%|          | 250/393931 [00:01<34:01, 192.84it/s]

  0%|          | 270/393931 [00:01<37:26, 175.23it/s]

  0%|          | 288/393931 [00:01<42:32, 154.25it/s]

  0%|          | 305/393931 [00:01<50:00, 131.20it/s]

  0%|          | 320/393931 [00:01<55:36, 117.96it/s]

  0%|          | 333/393

  1%|          | 2134/393931 [00:16<43:35, 149.77it/s]

  1%|          | 2151/393931 [00:16<43:37, 149.68it/s]

  1%|          | 2175/393931 [00:16<38:49, 168.19it/s]

  1%|          | 2196/393931 [00:16<36:46, 177.57it/s]

  1%|          | 2215/393931 [00:16<37:19, 174.92it/s]

  1%|          | 2234/393931 [00:16<36:58, 176.60it/s]

  1%|          | 2253/393931 [00:16<39:11, 166.56it/s]

  1%|          | 2273/393931 [00:16<37:18, 175.00it/s]

  1%|          | 2291/393931 [00:17<41:47, 156.19it/s]

  1%|          | 2308/393931 [00:17<42:19, 154.23it/s]

  1%|          | 2326/393931 [00:17<41:06, 158.77it/s]

  1%|          | 2343/393931 [00:17<50:49, 128.39it/s]

  1%|          | 2358/393931 [00:17<51:33, 126.57it/s]

  1%|          | 2376/393931 [00:17<47:13, 138.20it/s]

  1%|          | 2391/393931 [00:17<54:13, 120.34it/s]

  1%|          | 2411/393931 [00:18<47:50, 136.41it/s]

  1%|          | 2434/393931 [00:18<42:10, 154.71it/s]

  1%|          | 2453/393931 [00:18<42:41, 152.8

  1%|          | 4765/393931 [00:31<41:24, 156.63it/s]

  1%|          | 4782/393931 [00:31<42:30, 152.56it/s]

  1%|          | 4798/393931 [00:31<46:52, 138.38it/s]

  1%|          | 4813/393931 [00:32<46:06, 140.67it/s]

  1%|          | 4835/393931 [00:32<41:14, 157.22it/s]

  1%|          | 4858/393931 [00:32<37:21, 173.59it/s]

  1%|          | 4881/393931 [00:32<34:42, 186.84it/s]

  1%|          | 4901/393931 [00:32<36:30, 177.57it/s]

  1%|          | 4920/393931 [00:32<38:20, 169.13it/s]

  1%|▏         | 4938/393931 [00:32<39:15, 165.11it/s]

  1%|▏         | 4962/393931 [00:32<37:05, 174.79it/s]

  1%|▏         | 4980/393931 [00:32<37:02, 175.03it/s]

  1%|▏         | 4998/393931 [00:33<38:09, 169.85it/s]

  1%|▏         | 5022/393931 [00:33<34:51, 185.91it/s]

  1%|▏         | 5044/393931 [00:33<34:16, 189.09it/s]

  1%|▏         | 5064/393931 [00:33<34:52, 185.86it/s]

  1%|▏         | 5085/393931 [00:33<33:59, 190.70it/s]

  1%|▏         | 5107/393931 [00:33<32:54, 196.9

  2%|▏         | 7756/393931 [00:47<39:44, 161.92it/s]

  2%|▏         | 7774/393931 [00:47<43:11, 148.99it/s]

  2%|▏         | 7790/393931 [00:47<45:13, 142.32it/s]

  2%|▏         | 7805/393931 [00:47<55:02, 116.92it/s]

  2%|▏         | 7818/393931 [00:47<1:03:21, 101.58it/s]

  2%|▏         | 7830/393931 [00:47<1:07:38, 95.13it/s] 

  2%|▏         | 7841/393931 [00:47<1:06:20, 97.00it/s]

  2%|▏         | 7862/393931 [00:48<55:46, 115.37it/s] 

  2%|▏         | 7882/393931 [00:48<48:54, 131.57it/s]

  2%|▏         | 7905/393931 [00:48<42:42, 150.66it/s]

  2%|▏         | 7924/393931 [00:48<40:13, 159.95it/s]

  2%|▏         | 7950/393931 [00:48<36:14, 177.50it/s]

  2%|▏         | 7976/393931 [00:48<32:50, 195.82it/s]

  2%|▏         | 8004/393931 [00:48<29:59, 214.46it/s]

  2%|▏         | 8035/393931 [00:48<27:34, 233.27it/s]

  2%|▏         | 8061/393931 [00:48<27:49, 231.11it/s]

  2%|▏         | 8086/393931 [00:48<29:46, 215.99it/s]

  2%|▏         | 8109/393931 [00:49<31:01,

  3%|▎         | 10712/393931 [01:02<35:39, 179.10it/s]

  3%|▎         | 10731/393931 [01:02<37:58, 168.16it/s]

  3%|▎         | 10749/393931 [01:02<39:01, 163.65it/s]

  3%|▎         | 10767/393931 [01:02<38:36, 165.41it/s]

  3%|▎         | 10784/393931 [01:02<44:59, 141.93it/s]

  3%|▎         | 10808/393931 [01:02<39:40, 160.96it/s]

  3%|▎         | 10834/393931 [01:03<36:18, 175.82it/s]

  3%|▎         | 10855/393931 [01:03<34:37, 184.39it/s]

  3%|▎         | 10875/393931 [01:03<34:03, 187.46it/s]

  3%|▎         | 10895/393931 [01:03<37:00, 172.48it/s]

  3%|▎         | 10919/393931 [01:03<34:26, 185.32it/s]

  3%|▎         | 10939/393931 [01:03<33:50, 188.66it/s]

  3%|▎         | 10965/393931 [01:03<31:14, 204.25it/s]

  3%|▎         | 10987/393931 [01:03<31:17, 203.97it/s]

  3%|▎         | 11008/393931 [01:03<36:36, 174.34it/s]

  3%|▎         | 11032/393931 [01:04<33:38, 189.71it/s]

  3%|▎         | 11053/393931 [01:04<33:14, 191.99it/s]

  3%|▎         | 11074/393931 [

...

 98%|█████████▊| 384551/393931 [35:56<01:54, 82.05it/s]

 98%|█████████▊| 384560/393931 [35:56<02:00, 77.56it/s]

 98%|█████████▊| 384569/393931 [35:56<02:03, 75.77it/s]

 98%|█████████▊| 384579/393931 [35:56<01:54, 81.69it/s]

 98%|█████████▊| 384595/393931 [35:56<01:38, 94.36it/s]

 98%|█████████▊| 384609/393931 [35:56<01:31, 101.85it/s]

 98%|█████████▊| 384627/393931 [35:56<01:20, 115.38it/s]

 98%|█████████▊| 384645/393931 [35:57<01:12, 128.87it/s]

 98%|█████████▊| 384662/393931 [35:57<01:06, 138.43it/s]

 98%|█████████▊| 384678/393931 [35:57<01:06, 140.10it/s]

 98%|█████████▊| 384694/393931 [35:57<01:03, 145.22it/s]

 98%|█████████▊| 384710/393931 [35:57<01:08, 134.17it/s]

 98%|█████████▊| 384725/393931 [35:57<01:20, 114.71it/s]

 98%|█████████▊| 384738/393931 [35:57<01:22, 111.92it/s]

 98%|█████████▊| 384750/393931 [35:57<01:29, 102.51it/s]

 98%|█████████▊| 384761/393931 [35:58<02:22, 64.25it/s] 

 98%|█████████▊| 384770/393931 [35:58<02:31, 60.57it/s]

 98%|█████████▊| 384

 98%|█████████▊| 386667/393931 [36:12<00:54, 134.41it/s]

 98%|█████████▊| 386682/393931 [36:12<00:54, 131.88it/s]

 98%|█████████▊| 386697/393931 [36:12<00:53, 134.01it/s]

 98%|█████████▊| 386717/393931 [36:12<00:48, 148.60it/s]

 98%|█████████▊| 386733/393931 [36:12<00:50, 141.52it/s]

 98%|█████████▊| 386748/393931 [36:12<00:52, 136.87it/s]

 98%|█████████▊| 386775/393931 [36:12<00:45, 155.70it/s]

 98%|█████████▊| 386792/393931 [36:12<00:44, 159.20it/s]

 98%|█████████▊| 386809/393931 [36:13<00:47, 148.75it/s]

 98%|█████████▊| 386825/393931 [36:13<00:47, 150.72it/s]

 98%|█████████▊| 386841/393931 [36:13<00:48, 144.83it/s]

 98%|█████████▊| 386856/393931 [36:13<00:50, 139.72it/s]

 98%|█████████▊| 386875/393931 [36:13<00:46, 150.14it/s]

 98%|█████████▊| 386891/393931 [36:13<00:47, 148.25it/s]

 98%|█████████▊| 386908/393931 [36:13<00:45, 153.02it/s]

 98%|█████████▊| 386924/393931 [36:13<00:45, 153.92it/s]

 98%|█████████▊| 386940/393931 [36:13<00:45, 153.97it/s]

 98%|█████████

 99%|█████████▊| 388618/393931 [36:27<01:03, 83.10it/s]

 99%|█████████▊| 388629/393931 [36:27<00:59, 89.38it/s]

 99%|█████████▊| 388639/393931 [36:27<01:03, 83.92it/s]

 99%|█████████▊| 388648/393931 [36:27<01:02, 84.42it/s]

 99%|█████████▊| 388658/393931 [36:28<00:59, 88.51it/s]

 99%|█████████▊| 388673/393931 [36:28<00:52, 100.79it/s]

 99%|█████████▊| 388684/393931 [36:28<00:54, 95.94it/s] 

 99%|█████████▊| 388695/393931 [36:28<00:56, 93.22it/s]

 99%|█████████▊| 388705/393931 [36:28<00:55, 94.78it/s]

 99%|█████████▊| 388715/393931 [36:28<00:55, 93.74it/s]

 99%|█████████▊| 388726/393931 [36:28<00:53, 97.90it/s]

 99%|█████████▊| 388736/393931 [36:28<00:52, 98.28it/s]

 99%|█████████▊| 388746/393931 [36:28<00:53, 96.50it/s]

 99%|█████████▊| 388758/393931 [36:28<00:50, 102.14it/s]

 99%|█████████▊| 388769/393931 [36:29<00:52, 99.19it/s] 

 99%|█████████▊| 388780/393931 [36:29<00:56, 91.42it/s]

 99%|█████████▊| 388790/393931 [36:29<00:54, 93.67it/s]

 99%|█████████▊| 388803/393

 99%|█████████▉| 390174/393931 [36:43<00:25, 146.08it/s]

 99%|█████████▉| 390192/393931 [36:43<00:24, 153.90it/s]

 99%|█████████▉| 390209/393931 [36:43<00:24, 151.10it/s]

 99%|█████████▉| 390225/393931 [36:43<00:26, 140.29it/s]

 99%|█████████▉| 390240/393931 [36:43<00:26, 138.34it/s]

 99%|█████████▉| 390256/393931 [36:43<00:25, 143.53it/s]

 99%|█████████▉| 390280/393931 [36:44<00:22, 162.02it/s]

 99%|█████████▉| 390298/393931 [36:44<00:22, 163.06it/s]

 99%|█████████▉| 390322/393931 [36:44<00:20, 179.90it/s]

 99%|█████████▉| 390342/393931 [36:44<00:21, 163.15it/s]

 99%|█████████▉| 390367/393931 [36:44<00:19, 181.04it/s]

 99%|█████████▉| 390387/393931 [36:44<00:21, 165.34it/s]

 99%|█████████▉| 390406/393931 [36:44<00:20, 170.99it/s]

 99%|█████████▉| 390424/393931 [36:44<00:21, 159.68it/s]

 99%|█████████▉| 390441/393931 [36:44<00:21, 158.79it/s]

 99%|█████████▉| 390458/393931 [36:45<00:22, 156.58it/s]

 99%|█████████▉| 390479/393931 [36:45<00:20, 167.53it/s]

 99%|█████████

100%|█████████▉| 392396/393931 [36:58<00:10, 144.40it/s]

100%|█████████▉| 392412/393931 [36:58<00:11, 133.55it/s]

100%|█████████▉| 392427/393931 [36:59<00:12, 116.23it/s]

100%|█████████▉| 392440/393931 [36:59<00:13, 109.41it/s]

100%|█████████▉| 392456/393931 [36:59<00:12, 120.49it/s]

100%|█████████▉| 392469/393931 [36:59<00:12, 115.13it/s]

100%|█████████▉| 392484/393931 [36:59<00:11, 123.52it/s]

100%|█████████▉| 392498/393931 [36:59<00:11, 122.47it/s]

100%|█████████▉| 392515/393931 [36:59<00:10, 130.97it/s]

100%|█████████▉| 392539/393931 [36:59<00:09, 141.45it/s]

100%|█████████▉| 392556/393931 [37:00<00:09, 147.47it/s]

100%|█████████▉| 392572/393931 [37:00<00:09, 148.08it/s]

100%|█████████▉| 392593/393931 [37:00<00:08, 161.74it/s]

100%|█████████▉| 392610/393931 [37:00<00:08, 158.53it/s]

100%|█████████▉| 392629/393931 [37:00<00:07, 166.12it/s]

100%|█████████▉| 392647/393931 [37:00<00:08, 160.06it/s]

100%|█████████▉| 392664/393931 [37:00<00:08, 150.56it/s]

100%|█████████

In [18]:
if os.path.isfile('final.sqlite'):
    conn = sqlite3.connect('final.sqlite')
    final = pd.read_sql_query(""" SELECT * FROM Reviews """, conn)
    conn.close()
else:
    print("Please the above cell")

In [19]:
texts = list(final['CleanedText'])

In [20]:
print(type(texts))
print(type(texts[0]))
print(len(texts))

<class 'list'>
<class 'str'>
393931


In [21]:
outfile3 = open("Texts.pkl", "wb")
pickle.dump(texts, outfile3)

# [7.2.2] Bag of Words (BoW)

In [22]:
#BoW
count_vect = CountVectorizer() #in scikit-learn
final_counts = count_vect.fit_transform(final['CleanedText'].values)
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (393931, 74578)
the number of unique words  74578


## [7.2.4] Bi-Grams and n-Grams.

**Motivation**

Now that we have our list of words describing positive and negative reviews lets analyse them.<br>

We begin analysis by getting the frequency distribution of the words as shown below

In [23]:
with open('positive_words.pkl', 'rb') as f:
    all_positive_words = pickle.load(f)
with open('negitive_words.pkl', 'rb') as f:
    all_negative_words = pickle.load(f)
with open('neutral_words.pkl', 'rb') as f:
    all_neutral_words = pickle.load(f)
    
freq_dist_positive=nltk.FreqDist(all_positive_words)
freq_dist_negative=nltk.FreqDist(all_negative_words)
freq_dist_neutral=nltk.FreqDist(all_neutral_words)
print("Most Common Positive Words : ",freq_dist_positive.most_common(20))
print("Most Common Negative Words : ",freq_dist_negative.most_common(20))
print("Most Common Neutral Words : ",freq_dist_neutral.most_common(20))

Most Common Positive Words :  [(b'like', 139412), (b'tast', 129033), (b'good', 112759), (b'flavor', 109612), (b'love', 107349), (b'use', 103885), (b'great', 103865), (b'one', 96724), (b'product', 91031), (b'tri', 86786), (b'tea', 83885), (b'coffe', 78813), (b'make', 75105), (b'get', 72123), (b'food', 64800), (b'would', 55562), (b'time', 55264), (b'buy', 54196), (b'realli', 52713), (b'eat', 52001)]
Most Common Negative Words :  [(b'tast', 34575), (b'like', 32324), (b'product', 28215), (b'one', 20563), (b'flavor', 19571), (b'would', 17971), (b'tri', 17752), (b'use', 15301), (b'good', 15041), (b'coffe', 14707), (b'get', 13786), (b'buy', 13751), (b'order', 12869), (b'food', 12754), (b'dont', 11877), (b'tea', 11662), (b'even', 11085), (b'box', 10843), (b'amazon', 10073), (b'make', 9839)]
Most Common Neutral Words :  [(b'like', 20780), (b'tast', 20055), (b'flavor', 15951), (b'good', 13264), (b'product', 12218), (b'one', 11158), (b'would', 10924), (b'use', 10509), (b'coffe', 10239), (b'tri', 

<b>Observation:-</b> From the above it can be seen that the most common positive and the negative words overlap for eg. 'like' could be used as 'not like' etc. <br>
So, it is a good idea to consider pairs of consequent words (bi-grams) or q sequnce of n consecutive words (n-grams)

In [24]:
#bi-gram, tri-gram and n-gram

#removing stop words like "not" should be avoided before building n-grams
count_vect = CountVectorizer(ngram_range=(1,2) ) #in scikit-learn
final_bigram_counts = count_vect.fit_transform(final['CleanedText'].values)
print("the type of count vectorizer ",type(final_bigram_counts))
print("the shape of out text BOW vectorizer ",final_bigram_counts.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_bigram_counts.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (393931, 3105031)
the number of unique words including both unigrams and bigrams  3105031


# [7.2.5] TF-IDF

In [25]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['CleanedText'].values)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (393931, 3105031)
the number of unique words including both unigrams and bigrams  3105031


In [26]:
features = tf_idf_vect.get_feature_names()
print("some sample features(unique words in the corpus)",features[100000:100010])

some sample features(unique words in the corpus) ['anna make', 'anna never', 'anna nicol', 'anna one', 'anna orecciett', 'anna pappardell', 'anna pasta', 'anna pizza', 'anna power', 'anna save']


In [27]:
# source: https://buhrmann.github.io/tfidf-analysis.html
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [28]:
top_tfidf

,feature,tfidf
0,poem,0.383469
1,handmot invent,0.239689
2,poem throughout,0.239689
3,like handmot,0.239689
4,invent poem,0.239689
5,learn poem,0.239689
6,throughout school,0.239689
7,handmot,0.239689
8,way children,0.227094
9,learn month,0.227094


# [7.2.6] Word2Vec

In [29]:
# Using Google News Word2Vectors

# in this project we are using a pretrained model by google
# its 3.3G file, once you load this into your memory 
# it occupies ~9Gb, so please do this step only if you have >12G of ram
# we will provide a pickle file wich contains a dict , 
# and it contains all our courpus words as keys and  model[word] as values
# To use this code-snippet, download "GoogleNews-vectors-negative300.bin" 
# from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# it's 1.9GB in size.


# http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.W17SRFAzZPY
# you can comment this whole cell
# or change these varible according to your need
is_your_ram_gt_16g=False
want_to_read_sub_set_of_google_w2v = True
want_to_read_whole_google_w2v = True
if not is_your_ram_gt_16g:
    if want_to_read_sub_set_of_google_w2v and  os.path.isfile('google_w2v_for_amazon.pkl'):
        with open('google_w2v_for_amazon.pkl', 'rb') as f:
            # model is dict object, you can directly access any word vector using model[word]
            model = pickle.load(f)
else:
    if want_to_read_whole_google_w2v and os.path.isfile('GoogleNews-vectors-negative300.bin'):
        model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

# print("the vector representation of word 'computer'",model.wv['computer'])
# print("the similarity between the words 'woman' and 'man'",model.wv.similarity('woman', 'man'))
# print("the most similar words to the word 'woman'",model.wv.most_similar('woman'))
# this will raise an error
# model.wv.most_similar('tasti')  # "tasti" is the stemmed word for tasty, tastful

In [30]:
# Train your own Word2Vec model using your own text corpus
i=0
list_of_sent=[]
for sent in final['CleanedText'].values:
    list_of_sent.append(sent.split())

In [31]:
print(final['CleanedText'].values[0])
print("*****************************************************************")
print(list_of_sent[0])

june saw charm group rose begin droop pep chicken soup sprinkl sprinkl twice sprinkl chicken soup rice great book teach children month year repetit phrase funni littl stori accompani pictur make ideal bedtim read isnt near good sendak book like wild thing boy didnt care still carri uniqu brand charm
*****************************************************************
['june', 'saw', 'charm', 'group', 'rose', 'begin', 'droop', 'pep', 'chicken', 'soup', 'sprinkl', 'sprinkl', 'twice', 'sprinkl', 'chicken', 'soup', 'rice', 'great', 'book', 'teach', 'children', 'month', 'year', 'repetit', 'phrase', 'funni', 'littl', 'stori', 'accompani', 'pictur', 'make', 'ideal', 'bedtim', 'read', 'isnt', 'near', 'good', 'sendak', 'book', 'like', 'wild', 'thing', 'boy', 'didnt', 'care', 'still', 'carri', 'uniqu', 'brand', 'charm']


In [32]:
# min_count = 5 considers only words that occured atleast 5 times
w2v_model=Word2Vec(list_of_sent,min_count=5,size=50, workers=4)

In [33]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  22816
sample words  ['sempio', 'poodl', 'rotor', 'separat', 'unawar', 'bobbi', 'tet', 'schmaltz', 'linolen', 'poser', 'manx', 'development', 'organg', 'botan', 'sluggish', 'loveem', 'grouchi', 'octagon', 'left', 'coffemak', 'lebkuchen', 'romanoff', 'newport', 'anywho', 'mightv', 'shant', 'polan', 'panthothen', 'behav', 'protrud', 'fod', 'spaghettini', 'gabriel', 'iherb', 'keiller', 'gratif', 'girli', 'telma', 'fed', 'assert', 'bernard', 'tblsp', 'chanterell', 'molino', 'melba', 'peppi', 'pistachio', 'miser', 'sweater', 'teekann']


In [34]:
w2v_model.wv.most_similar('tasti')

[('delici', 0.7940857410430908),
 ('yummi', 0.7803266048431396),
 ('tastey', 0.7368853092193604),
 ('satisfi', 0.7012151479721069),
 ('good', 0.6943483352661133),
 ('nice', 0.6585299968719482),
 ('hearti', 0.6498652696609497),
 ('nutriti', 0.6358836889266968),
 ('delish', 0.6356318593025208),
 ('terrif', 0.6280478239059448)]

In [35]:
w2v_model.wv.most_similar('like')

[('alright', 0.7377765774726868),
 ('weird', 0.7306230068206787),
 ('okay', 0.710498034954071),
 ('dislik', 0.6959275007247925),
 ('prefer', 0.6855043768882751),
 ('yucki', 0.6683098673820496),
 ('appeal', 0.6676250696182251),
 ('remind', 0.6500131487846375),
 ('funki', 0.6476739645004272),
 ('resembl', 0.6432019472122192)]

# [7.2.7] Avg W2V, TFIDF-W2V

In [36]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sent): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))



  0%|          | 0/393931 [00:00<?, ?it/s]

  0%|          | 1/393931 [00:00<15:32:22,  7.04it/s]

  0%|          | 3/393931 [00:00<13:04:58,  8.36it/s]

  0%|          | 5/393931 [00:00<11:14:34,  9.73it/s]

  0%|          | 7/393931 [00:00<11:29:40,  9.52it/s]

  0%|          | 8/393931 [00:00<11:48:17,  9.27it/s]

  0%|          | 10/393931 [00:00<11:09:16,  9.81it/s]

  0%|          | 11/393931 [00:01<12:55:30,  8.47it/s]

  0%|          | 12/393931 [00:01<14:27:38,  7.57it/s]

  0%|          | 13/393931 [00:01<13:30:42,  8.10it/s]

  0%|          | 14/393931 [00:01<15:27:19,  7.08it/s]

  0%|          | 15/393931 [00:02<39:14:14,  2.79it/s]

  0%|          | 17/393931 [00:02<31:26:48,  3.48it/s]

  0%|          | 18/393931 [00:02<25:51:45,  4.23it/s]

  0%|          | 19/393931 [00:02<23:00:37,  4.76it/s]

  0%|          | 20/393931 [00:03<20:12:38,  5.41it/s]

  0%|          | 22/393931 [00:03<17:08:19,  6.38it/s]

  0%|          | 24/393931 [00:03<15:34:46,  7.02it/s]

  0%|  

  0%|          | 243/393931 [00:28<17:01:14,  6.42it/s]

  0%|          | 245/393931 [00:28<15:03:47,  7.26it/s]

  0%|          | 246/393931 [00:28<14:50:25,  7.37it/s]

  0%|          | 248/393931 [00:28<12:24:09,  8.82it/s]

  0%|          | 250/393931 [00:28<10:43:26, 10.20it/s]

  0%|          | 252/393931 [00:29<15:22:38,  7.11it/s]

  0%|          | 255/393931 [00:29<13:11:26,  8.29it/s]

  0%|          | 257/393931 [00:29<10:59:26,  9.95it/s]

  0%|          | 259/393931 [00:29<10:30:39, 10.40it/s]

  0%|          | 261/393931 [00:29<9:21:20, 11.69it/s] 

  0%|          | 263/393931 [00:30<19:08:38,  5.71it/s]

  0%|          | 265/393931 [00:30<16:17:45,  6.71it/s]

  0%|          | 267/393931 [00:30<16:49:51,  6.50it/s]

  0%|          | 268/393931 [00:31<15:28:53,  7.06it/s]

  0%|          | 270/393931 [00:31<13:22:34,  8.17it/s]

  0%|          | 272/393931 [00:31<15:59:14,  6.84it/s]

  0%|          | 274/393931 [00:31<13:48:19,  7.92it/s]

  0%|          | 275/393931 [00

KeyboardInterrupt: 

In [35]:
file = open("avg_vectors.pkl", "wb")
pickle.dump(sent_vectors, file)

In [36]:
# S = ["abc def pqr", "def def def abc", "pqr pqr def"]
model = TfidfVectorizer()
tf_idf_matrix = model.fit_transform(final['CleanedText'].values)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [37]:
# TF-IDF weighted Word2Vec
tfidf_feat = model.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_of_sent): # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

100%|██████████| 364171/364171 [57:21<00:00, 105.83it/s] 


In [39]:
file = open("tfidf_word2vec.pkl", "wb")
pickle.dump(tfidf_sent_vectors, file)

In [40]:
print(len(tfidf_sent_vectors))
print(len(tfidf_sent_vectors[0]))

364171
50


In [44]:
print(tfidf_sent_vectors[0])

[ 0.40625752 -0.81734095  0.0425389  -0.35226797  0.6338093  -0.51426871
 -0.12992926  0.46224967  0.20409222  0.59952927  0.43253375  0.75103276
  0.73529532 -0.36751139 -0.09252613 -1.1817231  -0.43135102 -0.03229498
  0.30454943  0.28574399 -0.29858537  0.09550053 -0.27387753  0.35206369
 -1.20623755  0.28937278  0.04187837  0.02608621 -0.12195129  0.30926015
 -0.00511797  0.07584566 -0.35218409 -0.06157059 -0.15456523  0.07483617
  0.43150275 -0.13060042 -0.14413061  0.68845195  0.1366235   0.46621115
  0.80258109  0.32651883 -0.14880838  0.15333252  0.22700159  0.15260635
  0.14964119 -0.03531183]
